In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# @title Load Configuraiton 
# import hbqaconfig
kaggle=True
# conf = hbqaconfig.setEnv('local')

# Setting for Kaggle & TPU

In [3]:
# !pip install torch_xla

In [4]:
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp

In [5]:
conf = {'DATA_FOLDER': '/kaggle/input/',
 'PE_FOLDER': '/kaggle/input/',
 'QAGS_FOLDER': '/kaggle/working/',
 'DRS_FOLDER': '/kaggle/working/',
 'AGS_FOLDER': '/kaggle/working/',
 'REPORT_FOLDER': '/kaggle/working/',
 'CORPUS_SECTIONS_FOLDER': '/kaggle/working/',
 'CORPUS_CHAPTER_FOLDER': '/kaggle/working/'}

for k,v in conf.items(): print (k,":",v)

DATA_FOLDER : /kaggle/input/
PE_FOLDER : /kaggle/input/
QAGS_FOLDER : /kaggle/working/
DRS_FOLDER : /kaggle/working/
AGS_FOLDER : /kaggle/working/
REPORT_FOLDER : /kaggle/working/
CORPUS_SECTIONS_FOLDER : /kaggle/working/
CORPUS_CHAPTER_FOLDER : /kaggle/working/


In [6]:
# @title Select Processing Device
import torch

# # Detect and initialize TPU
# tpu_available = tf.config.experimental.list_logical_devices("TPU")
# if tpu_available:
#     print("TPU available")
# else:
#     print("No TPU available")

if kaggle:
    DEVICE = torch.device("cuda") #xm.xla_device(fold + 1)
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
    
print(DEVICE)

cuda


# Zeroshot
- T5
- Flan-T5
- GPT2
- DistilBERT
- RoBERTa
- Llama2 (with PEFT & LoRA)

In [7]:
predmodels_list = {"0" :"T5:t5", "1" : "Flan-T5:flant5", "2":"GPT2:gpt2","3":"DistilBERT:distilbert",
                   "4":"RoBERTa:roberta","5":"Llama2:llama2", "6":"BERTSquad:bert", 
                   "7":"LongFormer:longformer", "8":"DistilBERT512:distilbert512"}
# Longformer doesn't work with latest Transformer

predict_now=True
embed_now=True
compute_metrics_now=True

# Example Questions

In [8]:
q1 = "What predicament does Yudhishthira face, and how does he seek guidance to resolve it?"

ra1 = "Yudhishthira faces the predicament of being unable to support the Brahmanas who are following him as he departs for the forest. \
He seeks guidance to resolve this dilemma by approaching his priest, Dhaumya, and inquiring about the appropriate course of action."

ra1 = ra1.replace('\n','')

c1 = """Section III
"Vaisampayana said, 'Yudhishthira the son of Kunti, thus addressed by Saunaka, approached his priest and in the midst of his brothers said,
'The Brahmanas versed in the Vedas are following me who am departing for the forest. Afflicted with many calamities I am unable to support them.
I cannot abandon them, nor have I the power to offer them sustenance: Tell me, O holy one, what should be done by me in such a pass.'
"Vaisampayana said, 'After reflecting for a moment seeking to find out the (proper) course by his yoga powers, Dhaumya, that foremost of all virtuous men,
addressed Yudhishthira, in these words, 'In days of old, all living beings that had been created were sorely afflicted with hunger.
And like a father (unto all of them), Savita (the sun) took compassion upon them. And going first into the northern declension,
the sun drew up water by his rays, and coming back to the southern declension, stayed over the earth, with his heat centered in himself.
And while the sun so stayed over the earth, the lord of the vegetable world (the moon), converting the effects of the solar heat (vapours) into clouds
and pouring them down in the shape of water, caused plants to spring up. Thus it is the sun himself, who, drenched by the lunar influence, is transformed,
upon the sprouting of seeds, into holy vegetable furnished with the six tastes. And it is these which constitute the food of all creatures upon the earth.
Thus the food that supporteth the lives of creatures is instinct with solar energy, and the sun is, therefore, the father of all creatures.
Do thou, hence, O Yudhishthira, take refuge even in him. All illustrious monarchs of pure descent and deeds are known to have delivered their people by practising high asceticism.
The great Karttavirya, and Vainya and Nahusha, had all, by virtue of ascetic meditation preceded by vows, delivered their people from heavy afflictions.
Therefore, O virtuous one, as thou art purified by the acts do thou likewise, entering upon a file of austerities. O Bharata, virtuously support the regenerate ones.'
"Janamejaya said, 'How did that bull among the Kurus, king Yudhishthira, for the sake of the Brahmanas adore the sun of wonderful appearance?"
"Vaisampayana said, 'Listen attentively, O king, purifying thyself and withdrawing thy mind from every other thing. And, O king of kings, appoint thou a time.
I will tell thee everything in detail, And, O illustrious one, listen to the one hundred and eight names (of the sun)
as they were disclosed of old by Dhaumya to the high-souled son of Pritha. Dhaumya said, 'Surya, Aryaman, Bhaga, Twastri, Pusha, Arka, Savitri.
Ravi, Gabhastimat, Aja, Kala, Mrityu, Dhatri, Prabhakara, Prithibi, Apa, Teja, Kha, Vayu, the sole stay, Soma, Vrihaspati, Sukra, Budha, Angaraka,
Indra, Vivaswat, Diptanshu, Suchi, Sauri, Sanaichara, Brahma, Vishnu, Rudra, Skanda, Vaisravana, Yama, Vaidyutagni, Jatharagni, Aindhna, Tejasampati,
Dharmadhwaja, Veda-karttri, Vedanga, Vedavahana, Krita, Treta, Dwapara, Kali, full of every impurity, Kala, Kastha, Muhurtta, Kshapa, Yama, and Kshana;
Samvatsara-kara, Aswattha, Kalachakra, Bibhavasu, Purusha, Saswata, Yogin, Vyaktavyakta, Sanatana, Kaladhyaksha, Prajadhyaksha, Viswakarma, Tamounda,
Varuna, Sagara, Ansu, Jimuta, Jivana, Arihan, Bhutasraya, Bhutapati, Srastri, Samvartaka, Vanhi, Sarvadi, Alolupa, Ananta, Kapila, Bhanu, Kamada,
Sarvatomukha, Jaya, Visala, Varada, Manas, Suparna, Bhutadi, Sighraga, Prandharana, Dhanwantari, Dhumaketu, Adideva, Aditisuta, Dwadasatman, Aravindaksha,
Pitri, Matri, Pitamaha, Swarga-dwara, Prajadwara, Mokshadwara, Tripistapa, Dehakarti, Prasantatman, Viswatman, Viswatomukha, Characharatman, Sukhsmatman,
the merciful Maitreya. These are the hundred and eight names of Surya of immeasurable energy, as told by the self-create (Brahma). For the acquisition of prosperity,
I bow down to thee, O Bhaskara, blazing like unto gold or fire, who is worshipped of the gods and the Pitris and the Yakshas, and who is adored by Asuras, Nisacharas,
and Siddhas. He that with fixed attention reciteth this hymn at sunrise, obtaineth wife and offspring and riches and the memory of his former existence,
and by reciting this hymn a person attaineth patience and memory. Let a man concentrating his mind, recite this hymn. By doing so,
he shall be proof against grief and forest-fire and ocean and every object of desire shall be his.'
"Vaisampayana continued, 'Having heard from Dhaumya these words suitable to the occasion, Yudhishthira the just, with heart concentrated within itself and purifying it duly,
became engaged in austere meditation, moved by the desire of supporting the Brahmanas. And worshipping the maker of day with offerings of flowers and other articles,
the king performed his ablutions. And standing in the stream, he turned his face towards the god of day. And touching the water of the Ganges
the virtuous Yudhishthira with senses under complete control and depending upon air alone for his sustenance, stood there with rapt soul engaged in pranayama.
And having purified himself and restrained his speech, he began to sing the hymn of praise (to the sun).'
'Yudhishthira said, "Thou art, O sun, the eye of the universe. Thou art the soul of all corporeal existences. Thou art the origin of all things.
Thou art the embodiment of the acts of all religious men. Thou art the refuge of those versed in the Sankhya philosophy (the mysteries of the 1."""

c1 = c1.replace('\n','')

In [9]:
q2 = "What is the significance of performing the Agnihotra, and what consequences are mentioned for neglecting it?"

ra2 = """Performing the Agnihotra is considered important for eternal morality. Neglecting it, as mentioned in the text, leads to the consumption of sin.
Those who do not perform the Agnihotra are said to suffer the consequences, including not waiting upon bulls and neglecting their kinsmen, guests, friends, sons, wives, and servants."""

ra2 = ra2.replace('\n','')

c2 = """Even this is eternal morality. They that perform not the Agnihotra  not wait upon bulls, nor cherish their kinsmen and guests and friends and
sons and wives and servants, are consumed with sin for such neglect. None should cook his food for himself alone and none should slay an animal without dedicating it to the gods,
the pitris, and guests. Nor should one eat of that food which hath not been duly dedicated to the gods and pitris. By scattering food on the earth, morning and evening,
for (the behoof of) dogs and Chandalas and birds, should a person perform the Viswedeva sacrifice.  He that eateth the Vighasa, is regarded as eating ambrosia.
What remaineth in a sacrifice after dedication to the gods and the pitris is regarded as ambrosia; and what remaineth after feeding the guest is called Vighasa
and is equivalent to ambrosia itself. Feeding a guest is equivalent to a sacrifice, and the pleasant looks the host casteth upon the guest, the attention he devoteth to him,
the sweet words in which he addresseth him, the respect he payeth by following him, and the food and drink with which he treateth him, are the five Dakshinas
in that sacrifice. He who giveth without stint food to a fatigued wayfarer never seen before, obtaineth merit that is great, and he who leading a domestic life,
followeth such practices, acquireth religious merit that is said to be very great. O Brahmana, what is thy opinion on this?"
"Saunaka said, 'Alas, this world is full of contradictions! That which shameth the good, gratifieth the wicked! Alas, moved by ignorance and passion
and slaves of their own senses, even fools perform many acts of (apparent merit) to gratify in after-life their appetites! With eyes open are these
men led astray by their seducing senses, even as a charioteer, who hath lost his senses, by restive and wicked steeds! When any of the six senses
findeth its particular object, the desire springeth up in the heart to enjoy that particular object. And thus when one's heart proceedeth to enjoy the
objects of any particular sense a wish is entertained which in its turn giveth birth to a resolve. And finally, like unto an insect falling into a flame from
love of light, the man falleth into the fire of temptation, pierced by the shafts of the object of enjoyment discharged by the desire constituting the seed of the resolve!
And thenceforth blinded by sensual pleasure which he seeketh without stint, and steeped in dark ignorance and folly which he mistaketh for a state of happiness, he
knoweth not himself! And like unto a wheel that is incessantly rolling, every creature, from ignorance and deed and desire, falleth into various states in this world,
wandering from one birth to another, and rangeth the entire circle of existences from a Brahma to the point of a blade of grass, now in water, now on land, and now against in the air!
'This then is the career of those that are without knowledge. Listen now to the course of the wise they that are intent on profitable virtue,
and are desirous of emancipation! The Vedas enjoin act but renounce (interest in) action. Therefore, shouldst thou act, renouncing Abhimana,  performance of sacrifices, study (of the Vedas),
gifts, penance, truth (in both speech and act), forgiveness, subduing the senses, and renunciation of desire,--these have been declared to be the eight (cardinal) duties constituting the true path.
Of these, the four first pave the way to the world of the pitris. And these should be practised without Abhimana. The four last are always observed by the pious, to attain the heaven of the gods.
And the pure in spirit should ever follow these eight paths. Those who wish to subdue the world for purpose of salvation, should ever act fully renouncing motives, effectually subduing their senses,
rigidly observing particular vows, devotedly serving their preceptors, austerely regulating their fare, diligently studying the Vedas, renouncing action as mean and restraining their hearts.
By renouncing desire and aversion the gods have attained prosperity. It is by virtue of their wealth of yoga  that the Rudras, and the Sadhyas, and the Adityas and the Vasus, and the twin Aswins,
rule the creatures. Therefore, O son of Kunti, like unto them, do thou, O Bharata, entirely refraining from action with motive, strive to attain success in yoga and by ascetic austerities.
Thou hast already achieved such success so far as thy debts to thy ancestors, both male and female concerned, and that success also which is derived from action (sacrifices). Do thou,
for serving the regenerate ones endeavour to attain success in penances. Those that are crowned with ascetic success, can, by virtue of that success, do whatever they list; do thou, therefore,
practising asceticism realise all thy wishes."""

c2 = c2.replace('\n','')

In [10]:
# # @title huggingface login
# !pip install huggingface-cli

# !huggingface-cli login

# Install Necessary Libararies

In [11]:
!pip install -Uq transformers
!pip install -Uq evaluate
!pip install -Uq SentencePiece
!pip install -Uq sentence-transformers
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2d50e56299958b347faf20cdf6f1e04945d3b9d0fab8124db8425b197f14ebbf
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [12]:
# !pip install accelerate>=0.20.1
# !pip install transformers[torch]
# # You need to restart the kernel after this step

# Load Libraries and Configurations

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import ast
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F

from sentence_transformers import SentenceTransformer, util

import tensorflow as tf

# import spacy
# import string

from sklearn.model_selection import train_test_split

import transformers
from transformers import pipeline
import evaluate  # Bleu


import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load QA Dataset

In [14]:
# df = pd.read_csv(datapath + '06-HBQA_Manual_with_Chunk.csv') # for local machine/vedavit colab

# Uncomment below code
# !pip install -q gdown
# import gdown

# # Replace the shared link with the actual link to your file
# file_url = 'https://drive.google.com/uc?id=1Euvnmp8yJ2LGlL2uDvDjER87PYz9RVvS'
# output_path = '/content/hbqa-colab.txt'  # Specify the desired file name and path

# gdown.download(file_url, output_path, quiet=False)

file='06-HBQA_Manual_with_Chunk.csv'
if kaggle: file = file.split(".")[0].lower().replace('_','-') +'/' + file
df = pd.read_csv(conf['PE_FOLDER'] + file)

In [15]:
print(df.shape)
df.head(2)

(1102, 11)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [16]:
df.set_index('Ques_Id',inplace=True)
df.head(2)

,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
Ques_Id,,,,,,,,,,
0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [17]:

# df['WordsInChunk'] = df.Chunk.str.split(' ').apply(len)
# df['CharInChunk'] = df.Chunk.apply(len)
# df.to_csv(r'H:\My Drive\HBQA\Data\06-HBQA_Manual_with_Chunk.csv')

In [18]:
# sample code
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# TOKENIZER = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# MODEL = AutoModelForQuestionAnswering.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# MODEL.to(DEVICE)

# Common Setting for Training

In [19]:
Question_Len = int(max(len(ques) for ques in df.Question)/4)
Answer_Len = int(max(len(ans) for ans in df.Ref_Answer)/4)
Chunk_Len = int(max(len(chunk) for chunk in df.Chunk)/4)
Chunk_Len, Question_Len,Answer_Len # in Tokens

(1499, 51, 326)

In [20]:
Q_LEN =  1500 #256   # Question Length
T_LEN =  500 #32  # Target Length
BATCH_SIZE = 4
# DEVICE = "cuda:0"

# Load Zeroshot Prediction Model

In [87]:
# @title Function to Load Zeroshot Transformers/Models
def load_model(predmodel_name):
    if predmodel_name=="t5":
        from transformers import T5Tokenizer,  T5ForConditionalGeneration #, T5Model , T5TokenizerFast
        tokenizer = T5Tokenizer.from_pretrained("t5-base")
        model = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
        model.to(DEVICE)
    elif predmodel_name == "roberta":
        from transformers import AutoTokenizer, AutoModelForQuestionAnswering
        model_name = "deepset/roberta-base-squad2"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        model.to(DEVICE)

    elif predmodel_name=="gpt2":
        from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
        tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium",  padding_side="left")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
        tokenizer.pad_token = tokenizer.eos_token
        model = GPT2LMHeadModel.from_pretrained("gpt2-medium").to(DEVICE)  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl

    elif predmodel_name=="bert":
        from transformers import AutoTokenizer, AutoModelForQuestionAnswering
        model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
        # model_name = "t5-base"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        model.to(DEVICE)
    elif predmodel_name=="longformer":
        print ("It doesnot work with new transformers")
        #     from transformers import LongformerTokenizer, LongformerForQuestionAnswering
        #     model_name = "valhalla/longformer-base-4096-finetuned-squadv1"
        #     tokenizer = LongformerTokenizer.from_pretrained(model_name) #.to(DEVICE)
        #     model = LongformerForQuestionAnswering.from_pretrained(model_name).to(DEVICE)

    elif predmodel_name=="distilbert":
        # from transformers import DistilBertTokenizer, DistilBertModel
        # tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
        # model = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')
        # model.to(DEVICE)

        from transformers import AutoTokenizer, AutoModelForQuestionAnswering
        model_name = 'distilbert-base-cased-distilled-squad'
        tokenizer = AutoTokenizer.from_pretrained(model_name) #.to(DEVICE)
        model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(DEVICE)
    elif predmodel_name=="distilbert512":   
        from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
        model_name = 'distilbert-base-cased-distilled-squad'
        tokenizer = DistilBertTokenizer.from_pretrained(model_name)
        model = DistilBertForQuestionAnswering.from_pretrained(model_name).to(DEVICE)


    return tokenizer, model

In [29]:
predmodels_list # done: 0,3,6,8

{'0': 'T5:t5',
 '1': 'Flan-T5:flant5',
 '2': 'GPT2:gpt2',
 '3': 'DistilBERT:distilbert',
 '4': 'RoBERTa:roberta',
 '5': 'Llama2:llama2',
 '6': 'BERTSquad:bert',
 '7': 'LongFormer:longformer',
 '8': 'DistilBERT512:distilbert512'}

In [74]:
# @title Select prediction model
predmodel_name = predmodels_list['2'].split(':')[1]
print(predmodel_name)
tokenizer, model = load_model(predmodel_name)

gpt2


# Function to predict Answers

In [77]:
# @title answer_from_long_sentence_distilbert
def answer_from_long_sentence_distilbert(inputs,model):
  # Check if the input sequence length exceeds the model's maximum
  if len(inputs["input_ids"][0]) > 512:
      # Split the input into segments that fit the model's maximum sequence length
      input_ids = inputs["input_ids"][0]
      attention_mask = inputs["attention_mask"][0]

      # Create a tensor with zeros
      pad_len = 512 - len(input_ids) % 512
      zeros = torch.zeros(pad_len, dtype=input_ids.dtype).to(DEVICE)

      # Append the zeros to the input_ids tensor and attention_mask
      input_ids = torch.cat((input_ids, zeros))
      attention_mask = torch.cat((attention_mask, zeros)).to(DEVICE)

      start_positions = []
      end_positions = []

      while len(input_ids) >= 512:
          # Extract the first 512 tokens
          segment_input_ids = input_ids[:512]
          segment_attention_mask = attention_mask[:512]
          # print(len(segment_input_ids))

          with torch.no_grad():
              segment_outputs = model(input_ids=segment_input_ids, attention_mask=segment_attention_mask)

          start_logits = segment_outputs.start_logits
          end_logits = segment_outputs.end_logits

          # Find the answer span within the segment
          answer_start = torch.argmax(start_logits)
          answer_end = torch.argmax(end_logits) + 1

          # Add the segment positions to the overall positions
          start_positions.append(answer_start)
          end_positions.append(answer_end)

          # Remove the processed tokens
          input_ids = input_ids[512:]
          attention_mask = attention_mask[512:]

      # Determine the final answer based on the segment positions
      answer_start = min(start_positions)
      answer_end = max(end_positions)
  else:
      # Use the original answer extraction approach
      with torch.no_grad():
          outputs = model(**inputs)
      answer_start = torch.argmax(outputs.start_logits)
      answer_end = torch.argmax(outputs.end_logits) + 1


  # Get the answer text from the input tokens
  answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

  if predicted_answer[0:5]=='[CLS]' or  predicted_answer[0:5]=='[SEP]' or predicted_answer[0:3]=='<s>' or len(predicted_answer)<3 :
     predicted_answer="xxx"

  return answer

In [79]:
Q_LEN

1500

In [98]:
# @title Predict Answer Function
def predict_answer(context, question, tokenizer, model):
    Q_LEN=1500
    
    if predmodel_name=="t5":
        inputs = tokenizer(question, context, max_length= Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

        input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
        attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

        predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)
    elif predmodel_name=="roberta":
        inputs = tokenizer(question, context, return_tensors="pt", padding=True, max_length=512, truncation="only_second").to(DEVICE)

        with torch.no_grad():
            outputs = model(**inputs)

        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)+50
        predicted_answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end+1])
        
    elif predmodel_name=="gpt2":
#         Q_LEN=1024
        # inputs = tokenizer(context, question, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
        # input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
        # attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)
        # outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

        # Tokenize the input and set pad_token_id

        input_text = f"{context} [SEP] Q: {question} A:"
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=Q_LEN, 
                                   truncation=True, padding="max_length", pad_to_max_length=True).to(DEVICE)

        # Set pad_token_id to eos_token_id
        model.config.pad_token_id = model.config.eos_token_id

        # Generate text
        with torch.no_grad():
            outputs = model.generate(input_ids, max_length=20, num_return_sequences=1, no_repeat_ngram_size=2)

        predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elif predmodel_name=="longformer":
        print("It does not work. So removed")

    elif predmodel_name=="bert":
      # inputs = tokenizer(question, context, return_tensors="pt", padding=True, max_length=1500, truncation="only_second")
      # predicted_answer = answer_from_long_sentence_bert(inputs,model)

      inputs = tokenizer(question, context, return_tensors="pt", padding=True, max_length=512, truncation=True).to(DEVICE)
      with torch.no_grad():
          outputs = model(**inputs)


      start_scores = outputs.start_logits
      end_scores = outputs.end_logits

      answer_start = torch.argmax(start_scores)
      answer_end = torch.argmax(end_scores)+50
      predicted_answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end+1])

    elif predmodel_name=="distilbert":
      inputs = tokenizer(question, context, return_tensors="pt", padding=True, max_length=1500, truncation="only_second").to(DEVICE)

      predicted_answer = answer_from_long_sentence_distilbert(inputs,model)
    elif predmodel_name=="distilbert512":
      inputs = tokenizer(question, context, return_tensors="pt", padding=True, max_length=512, truncation="only_second").to(DEVICE)

      with torch.no_grad():
          outputs = model(**inputs)

      start_scores = outputs.start_logits
      end_scores = outputs.end_logits

      answer_start = torch.argmax(start_scores)
      answer_end = torch.argmax(end_scores)+50
      predicted_answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end+1])


    if len(predicted_answer)<3:
       predicted_answer="xxx" 
    elif predicted_answer[0:5]=='[CLS]' or  predicted_answer[0:5]=='[SEP]' or predicted_answer[0:3]=='<s>' :
        predicted_answer="xxx"
    return predicted_answer

In [99]:
# @title Example Prediction with a Selcted Model

# from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
# inputs = tokenizer(q2, c2, return_tensors="pt", padding=True, max_length=1500, truncation="only_second")

# pred_answer = predict_answer(c2, q2, tokenizer,model)
# print(pred_answer)

In [100]:
# inputs = tokenizer(q2, c2, return_tensors="pt", padding=True, max_length=512, truncation=True)
# with torch.no_grad():
#     outputs = model(**inputs)

In [101]:
# inputs.keys()

In [102]:
# type(outputs)
# outputs.keys()

In [103]:
# @title Predict Answer Function
from IPython.display import clear_output
clear_output()

from IPython.display import display
from IPython.display import HTML

if predict_now:
  df_pred = pd.DataFrame(columns=['Ques_Id','Question','Ref_Answer','Pred_Answer'])

def predict_all_answers(df_pred, sample=True, verbose=False):
  import random
  if sample:
    qno=random.sample(set(df.index),10)
  else:
    qno = df.index

  total_items = len(qno)
  j=0

  # progress_bar = tqdm(total=total_items, position=0, leave=True)

  progress_bar = tqdm(total=total_items, position=0, leave=True, dynamic_ncols=True)
  progress_html = HTML("""<progress value="0" max="{0}" style="width: 100%"></progress>""".format(total_items))


  for i  in qno:
      progress_bar.update(j)
      progress_html.value = """<progress value="{0}" max="{1}" style="width: 100%"></progress>""".format(i, total_items)
      progress_bar = tqdm(total=total_items, position=0, leave=True)
      ques_id  = df.index[i]
      chunk    = df.iloc[i]['Chunk']
      # print(chunk)
      # chunk = chunk.replace("'","").replace("\","")
      ques     = df.iloc[i]['Question']
      ref_ans  = df.iloc[i]['Ref_Answer']

      pred_ans = predict_answer(chunk, ques, tokenizer, model)

      df_pred.loc[j] = ques_id, ques, ref_ans, pred_ans
      j+=1
      if verbose:
        print('Question  :', ques)
        print("Ref Answer:", ref_ans)
        print("Pred Ans  :", pred_ans)
        print('--------')

      # print(f"Prediting Answer {i}/{df.shape[0]}")

  # progress_bar.close()

In [104]:
# @title Start Predicting Answer
filenm = '09.11-' + predmodel_name +'Predicted_Ans-Zeroshot.csv'
if predict_now:
  predict_all_answers(df_pred, sample=False, verbose=True)

if predict_now:
  print('Saving Predictions')
  df_pred.to_csv(conf['AGS_FOLDER'] + filenm, index=False)

  0%|          | 0/1102 [00:00<?, ?it/s]
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [34,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [34,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [34,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [34,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [34,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
df_pred.to_csv(conf['AGS_FOLDER'] + filenm, index=False)

In [41]:

# df_pred = pd.read_csv(conf['AGS_FOLDER'] + filenm)
df_pred.shape


(1102, 4)

In [42]:
# @title Load Predicted Asnwer file
# if kaggle: file = filenm.split(".")[0].lower().replace('_','-') +'/' + file
df_pred = pd.read_csv(conf['AGS_FOLDER'] + filenm)
print(df_pred.shape)
df_pred.head(2)

(1102, 4)


,Ques_Id,Question,Ref_Answer,Pred_Answer
0,0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,xxx
1,1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"hunger. And like a father (unto all of them),..."


# Load Embedding Model

In [43]:
#Select Model Function

# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
#290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
#1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

def select_embmodel(num):
    emb_modelshortlist = ['distilbert','minilm','distilroberta','mpnet','roberta']

    emb_modellist = ['multi-qa-distilbert-cos-v1',
                'all-MiniLM-L6-v2',
                'all-distilroberta-v1',
                'multi-qa-mpnet-base-dot-v1',
                'all-roberta-large-v1']

    embmodelname = emb_modellist[num]
    embmodelshort = emb_modelshortlist[num]
    embmodelname1 = "_" + embmodelname

    print (embmodelname,'\t',embmodelshort,'\t', embmodelname1)
    return embmodelname, embmodelshort, embmodelname1

In [44]:
embmodelname, embmodelshort, embmodelname1 = select_embmodel(3)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


In [45]:
if embed_now:
  embmodel = SentenceTransformer(embmodelname)

# Vectorize Answer

In [46]:
filenm ='09.11-' + predmodel_name +'Predicted_Ans-Zeroshot.csv'
df_pred =  pd.read_csv(conf['AGS_FOLDER'] + filenm)

df_pred.set_index('Ques_Id',inplace=True)

In [47]:
df_pred.head(3)

,Question,Ref_Answer,Pred_Answer
Ques_Id,,,
0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,xxx
1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"hunger. And like a father (unto all of them),..."
2,What advice does Dhaumya offer to Yudhishthira...,Dhaumya advises Yudhishthira to take refuge in...,"Tell me, O holy one, what should be done by m..."


In [48]:
def create_ans_vector():

    Ans_Sentences = df_pred.Pred_Answer.tolist()
    Ans_Embeddings = embmodel.encode(Ans_Sentences)

    print ("Answer Embedding: ", Ans_Embeddings.shape)
    # return (Ques_Embeddings, Ans_Embeddings)

    return (Ans_Embeddings)

In [49]:
# @title Answer not found these questions
df_pred['Pred_Answer'] = df_pred.Pred_Answer.fillna("xxx")
df_pred.loc[df_pred['Pred_Answer'].str.contains(r'\?\?\?'), 'Pred_Answer'] = "xxx"

# Model couldn't fine any any answer for this
df_pred.loc[df_pred['Pred_Answer'].str.contains(r"xxx")].shape

(681, 3)

In [50]:
if embed_now:
  Ans_Embeddings = create_ans_vector()
  df_pred['Pred_Answer_Vector'] = torch.tensor(Ans_Embeddings, dtype=torch.float).tolist()

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Answer Embedding:  (1102, 768)


In [51]:

filenm = '09.11-' + predmodel_name +'Predicted_AnsVec-Zeroshot.csv'
if embed_now:
  df_pred.to_csv(conf['AGS_FOLDER'] + filenm)

# df_pred = pd.read_csv(datapath + '09.11-' + predmodel_name +'Predicted_AnsVec-Zeroshot.csv')
print(df_pred.shape)
df_pred.head(2)

(1102, 4)


,Question,Ref_Answer,Pred_Answer,Pred_Answer_Vector
Ques_Id,,,,
0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,xxx,"[-0.3637753129005432, -0.34021222591400146, -0..."
1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"hunger. And like a father (unto all of them),...","[0.19928711652755737, -0.4345826804637909, -0...."


# Calculate Metrics

In [52]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import precision_score, recall_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import numpy as np

smoother = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def get_nlp_metrics(ques_id, ref_ans, pred_ans):

  # Calculate ROUGE-1 and ROUGE-L scores
  scores = scorer.score(ref_ans, pred_ans)

  # Access individual ROUGE scores
  rouge_1_precision = scores['rouge1'].precision
  rouge_1_recall = scores['rouge1'].recall
  rouge_1_f1 = scores['rouge1'].fmeasure

  rouge_l_precision = scores['rougeL'].precision
  rouge_l_recall = scores['rougeL'].recall
  rouge_l_f1 = scores['rougeL'].fmeasure

  rouge_1_precision, rouge_1_recall, rouge_1_f1, rouge_l_precision, rouge_l_recall, rouge_l_f1,

  ref_tokens = word_tokenize(ref_ans)
  pred_tokens = word_tokenize(pred_ans)

  # Calculate BLEU score for a single sentence
  bleu_score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoother.method2)

  # Calculate BLEU score for multiple sentences
  # corpus_bleu_score = corpus_bleu([[ref_tokens]], [pred_tokens],  smoothing_function=smoother.method2)

  # print(set(ref_tokens).intersection(set(pred_tokens)))
  tp = len(set(ref_tokens).intersection(set(pred_tokens)))
  precision = tp / len(pred_tokens)
  recall =  tp / len(ref_tokens)

  return ques_id, bleu_score, rouge_1_precision, rouge_1_recall, rouge_1_f1, rouge_l_precision, rouge_l_recall, rouge_l_f1, precision, recall

In [53]:
# from datasets import load_metric
if compute_metrics_now:
  df_metrics = pd.DataFrame(columns = ['Ques_Id','BLEU1', 'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1',
                              'ROUGEL_P', 'ROUGEL_R', 'ROUGEL_F1', 'Precision', 'Recall'])
  N= df_pred.shape[0]

  for i in range(N):
    # bleu_score1 = calculate_score( df1.iloc[i]['ref_answer'],df1.iloc[i]['pred_answer'])
    ques_id = df_pred.index[i]
    ref_ans  = df_pred.loc[df_pred.index==ques_id,'Ref_Answer'].values[0]
    pred_ans = df_pred.loc[df_pred.index==ques_id,'Pred_Answer'].values[0]
    results = get_nlp_metrics(ques_id, ref_ans, pred_ans)
    df_metrics.loc[i] = results

In [54]:
metricsfile = '09.12-' + predmodel_name +'Predicted_Ans_Score_Zeroshot.csv'
print(metricsfile)

if compute_metrics_now:
  df_metrics.to_csv(conf['REPORT_FOLDER'] + metricsfile, index=False)

# df_metrics = pd.read_csv(datapath + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

09.12-robertaPredicted_Ans_Score_Zeroshot.csv
(1102, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGE1_F1,Precision,Recall
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.0,0.048441,0.236842,0.219512,0.227848,0.184211,0.170732,0.177215,0.166667,0.177778
2,2.0,0.045872,0.285714,0.235294,0.258065,0.166667,0.137255,0.150538,0.245283,0.216667


In [55]:
df_metrics = pd.read_csv(conf['REPORT_FOLDER'] + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

(1102, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGE1_F1.1,Precision,Recall
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.0,0.048441,0.236842,0.219512,0.227848,0.184211,0.170732,0.177215,0.166667,0.177778
2,2.0,0.045872,0.285714,0.235294,0.258065,0.166667,0.137255,0.150538,0.245283,0.216667


# Calculate Cosine between Predicted and Reference Answer

In [56]:
filenm = '07.2-HBQA_QA_Vector' + embmodelname1 + '.csv'


In [57]:
# filenm = r'07.2-HBQA_Cosine_multi-qa-mpnet-base-dot-v1.csv'
filenm = '07.2-HBQA_QA_Vector' + embmodelname1 + '.csv'
filenm = '07-2-hbqa-cosine-multi-qa-mpnet-base-dot-v1/07.2-HBQA_Cosine_multi-qa-mpnet-base-dot-v1.csv'
# if kaggle: filenm = filenm.split(".")[0].lower().replace('_','-') +'/' + filenm
    
df_ref_ans = pd.read_csv(conf['PE_FOLDER'] + filenm)

df_ref_pred= df_pred.merge(df_ref_ans, on = "Ques_Id", how="inner")[['Ques_Id','Pred_Answer_Vector','AnsVector_multi-qa-mpnet-base-dot-v1']]

In [58]:
def calculate_cosine(row):

    if type(row['Pred_Answer_Vector'])!=type(list()):
      predAns_vector_values = ast.literal_eval(row['Pred_Answer_Vector'])
    else:
      predAns_vector_values = row['Pred_Answer_Vector']

    if type(row['AnsVector' + embmodelname1])!=type(list()):
      refAns_vector_values = ast.literal_eval(row['AnsVector' + embmodelname1])
    else:
      refAns_vector_values = row['AnsVector' + embmodelname1]


    # Convert the string values to floats
    predAns_vector_values = [float(value) for value in predAns_vector_values]
    refAns_vector_values = [float(value) for value in refAns_vector_values]

    # Convert the lists to PyTorch tensors
    predAns_vector_values = torch.tensor(predAns_vector_values).reshape(1, -1)
    refAns_vector_values = torch.tensor(refAns_vector_values).reshape(1, -1)

    similarity = cosine_similarity(predAns_vector_values, refAns_vector_values)
    return similarity.item()

In [59]:
df_ref_pred['Cosine'] = df_ref_pred.apply(calculate_cosine, axis=1)

In [60]:
df_ref_pred.head(2)

,Ques_Id,Pred_Answer_Vector,AnsVector_multi-qa-mpnet-base-dot-v1,Cosine
0,0,"[-0.3637753129005432, -0.34021222591400146, -0...","[-0.316677063703537, 0.04548855870962143, -0.2...",0.240315
1,1,"[0.19928711652755737, -0.4345826804637909, -0....","[-0.013333199545741081, 0.05390685796737671, -...",0.429423


In [61]:
df_metrics = df_metrics.merge(df_ref_pred, on="Ques_Id", how="left")[['Ques_Id', 'BLEU1', 'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1', 'ROUGEL_P',
       'ROUGEL_R', 'ROUGE1_F1.1', 'Precision', 'Recall','Cosine']]

# Save File Metrics & Show Summary

In [62]:
metricsfile = '09.12-' + predmodel_name +'Predicted_Ans_Score_Zeroshot.csv'
print(metricsfile)
df_metrics.to_csv(conf['REPORT_FOLDER']+ metricsfile, index=False)
df_metrics.head(2)

09.12-robertaPredicted_Ans_Score_Zeroshot.csv


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGE1_F1.1,Precision,Recall,Cosine
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240315
1,1.0,0.048441,0.236842,0.219512,0.227848,0.184211,0.170732,0.177215,0.166667,0.177778,0.429423


In [69]:
all_metrics = pd.DataFrame()
all_metrics[predmodel_name] = df_metrics.mean()[1:]

In [70]:
all_metrics

,roberta
BLEU1,0.030853
ROUGE1_P,0.131937
ROUGE1_R,0.122343
ROUGE1_F1,0.119923
ROUGEL_P,0.088812
ROUGEL_R,0.082219
ROUGE1_F1.1,0.080415
Precision,0.096112
Recall,0.092252
Cosine,0.370905


In [71]:
all_metrics.to_csv("all_metrics1.csv")

In [ ]:
df_metrics.mean()

In [72]:
!zip -r file.zip /kaggle/working

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/09.11-robertaPredicted_Ans-Zeroshot.csv (deflated 69%)
  adding: kaggle/working/09.12-robertaPredicted_Ans_Score_Zeroshot.csv (deflated 74%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/all_metrics1.csv (deflated 38%)
  adding: kaggle/working/09.11-robertaPredicted_AnsVec-Zeroshot.csv (deflated 73%)


In [73]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip